In [40]:
import json

In [41]:
# run it for each post file
dates=['2021','2020','2019']
pages=['1','2','3']
array = []

for page in pages:
    for date in dates:
        with open(date + '_' + page + '_post.json', 'r') as f:
            data = json.load(f)
            comment_no = 0
            for key in data:
                if key == "comments":
                    for comment in data[key]:
                        row=[]
                        row.append(date)
                        row.append(page)
                        for i in range(0,11):
                            # if the data is in a list ie 1. 2. 3. 
                            try: 
                                value=data[key][comment]['media']["richtextContent"]["document"][0]['c'][i]['c'][0]['c'][0]['t']
                                if "Remote:" not in value:
                                    if "Internship:" not in value:
                                        if "Details:" not in value:
                                            row.append(value)
                            except:
                                # if the data is in a list, but has a non-list sentence first. (Posters often add a preamble)
                                try: 
                                    value=data[key][comment]['media']["richtextContent"]["document"][1]['c'][i]['c'][0]['c'][0]['t']
                                    if "Remote:" not in value:
                                        if "Internship:" not in value:
                                            if "Details:" not in value:
                                                row.append(value)

                                except:
                                    try: 
                                        # this works if the data is not in a list
                                        value=data[key][comment]['media']["richtextContent"]["document"][i]['c'][0]['t']
                                        if "Remote:" not in value:
                                            if "Internship:" not in value:
                                                if "Details:" not in value:
                                                    row.append(value)
                                    except:
                                        pass
                        # remove results with less than 6 lines - these tend to be comments that do not contain salary data (which has 5-7 lines)
                        if len(row)>5:
                            array.append(row)
                        comment_no += 1

In [43]:
import pandas as pd
df=pd.DataFrame(array)
df.columns=['date','page','title','tenure','location','salary','industry','education', 'prior_experience','signing_bonus','stock_or_bonus','total_comp', 'extra_col']
df.to_csv('salary_data.csv', index=False)

In [44]:
df

,date,page,title,tenure,location,salary,industry,education,prior_experience,signing_bonus,stock_or_bonus,total_comp,extra_col
0,2021,1,Title: Data science manager,• Tenure length: 2 years,• Location: fully remote,• Salary: $190k,• Company/Industry: ad tech,"• Education: PhD, computer science",• Prior Experience: 5 years as a data scientist,• Relocation/Signing Bonus: n/a,• Stock and/or recurring bonuses: profit shari...,• Total comp: $210k,None
1,2021,1,Title: Quantitative Analyst,Tenure length: < 6 months,Location: LCOL midwest city,Salary: $75k,Industry: Financial Services,Education: BS mathematics/MS applied statistics,"Prior experience: 2 quant internships, 1 unrel...",Signing bonus: $5k,Annual bonus: ~10% salary,Total comp: ~80-85k,None
2,2021,1,Title: Senior Predictive Analyst,Tenure length: 5 years in current role,"Location: Southern, USA (optional remote / hyb...","Salary: $93,000",Company/Industry: Financial Consulting,Education: BS Applied Mathematics; Minior in A...,Prior Experience: N/A,Relocation/Signing Bonus: None,Stock and/or recurring bonuses: None,"Total comp: $93,000",None
3,2021,1,Title: Senior Data Scientist,Tenure length: 3 years,Location: lcol midwest,"Salary: 115,000",Company/Industry: financial services,"Education: MS, MBA",Prior experience: BI,Relocation/Signing Bonus: no,Stock and/or recurring bonuses: 16% annual bon...,None,None
4,2021,1,• Title: Data Scientist,• Tenure length: 1.5 years,• Location: Washington DC,"• Salary: $88,000",• Company/Industry: National Security (governm...,"• Education: M.S. in Data Science, B.S. Politi...",• Prior Experience: Came straight out of grad ...,• Relocation/Signing Bonus: none,• Stock and/or recurring bonuses: none,• Total comp: 88k but I do make and additional...,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,2019,3,Title: Data Scientist,Tenure length: 4 years,Location: (US) Boonies,"Salary: 100,400",Company/Industry: Finance (just shy of 5k empl...,Education: masters in statistics,Prior Experience: None,Relocation/Signing Bonus: None,Stock and/or recurring bonuses: 7200,"Total comp: 107,600",None
307,2019,3,Title: Data Scientist,Tenure length: haven't started yet,"Location: Austin, TX",Salary: $65k,Company/Industry: Large automaker,Education: B.S. Comp Sci,Prior Experience:,"Relocation/Signing Bonus: not sure, but I know...",Stock and/or recurring bonuses: recurring 10% ...,Total comp: $85k,None
308,2019,3,Title: Marketing Coordinator,Tenure length: ~5 years,Location: Ohio,Salary: $30k,Company/Industry: Multifamily Housing,Education: BFA Ballet and BS Political Science,Prior Experience: None,Stock and/or recurring bonuses: None,Total comp: 30k,None,None
309,2019,3,Title: Senior Data Scientist,Tenure length: 8 months,Location: Denver,"Salary: $130,000",Company/Industry:,Education: PhD,Prior Experience: 3 years at 2 companies as a ...,Relocation/Signing Bonus: 1k relocation,Stock and/or recurring bonuses: 10% annual bon...,"Total comp: $130-143,000",None


In [39]:
#create sqlite database
import sqlite3
conn = sqlite3.connect('../df_final.db')

#write to sqlite db
df.to_sql('data_pre_clean', con=conn, if_exists='replace', index=False)

382

In [6]:

# upload to snowflake
import snowflake.connector
from sqlalchemy import create_engine
# from snowflake.connector.pandas_tools import write_pandas 
from snowflake.connector.pandas_tools import pd_writer
from snowflake.sqlalchemy import URL

In [7]:
snowflake_user='devadmin'
snowflake_account='vb69737.us-central1.gcp'
database='NEEDFUL_THINGS',
schema='PUBLIC'
warehouse='COMPUTE_WH'

In [8]:
# Credentials in 1Password
from getpass import getpass
snowflake_password = getpass('Enter the snowflake password (stored in 1password): ')

In [9]:
#snowflake is fussy
df.columns=map(str.upper, df.columns)

In [10]:

from snowflake.sqlalchemy import URL
from sqlalchemy import create_engine

engine = create_engine(URL(
    account = snowflake_account,
    user=snowflake_user,
    password=snowflake_password,
))

connection = engine.connect()
try:
    connection.execute('''USE DATABASE BLOG''')
    connection.execute('''USE WAREHOUSE COMPUTE_WH''')
    df.to_sql('raw_salary_data', connection, if_exists='replace', index = False)
finally:
    connection.close()
    engine.dispose()